# NCCS Chatbot 

In [13]:
from langchain.vectorstores import Pinecone
from langchain import OpenAI, VectorDBQA
from langchain.llms import HuggingFacePipeline
# from langchain.chains.question_answering import load_qa_chain
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from getpass import getpass
import pinecone

from embeddings import LocalHuggingFaceEmbeddings

In [6]:
# Initialise pinecone 

# find API key in console at app.pinecone.io
YOUR_API_KEY = getpass("Pinecone API Key: ")
# find ENV (cloud region) next to API key in console
YOUR_ENV = input("Pinecone environment: ")

index_name = 'langchain-retrieval-augmentation'
pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
)

index = pinecone.GRPCIndex(index_name)
index.describe_index_stats()

Pinecone API Key: ········
Pinecone environment: northamerica-northeast1-gcp


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 300}},
 'total_vector_count': 300}

In [14]:
embeddings = LocalHuggingFaceEmbeddings('multi-qa-mpnet-base-dot-v1')

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)     

# Local LLM - Koala 7b 8-bit

In [17]:
tokenizer = LlamaTokenizer.from_pretrained("samwit/koala-7b")

base_model = LlamaForCausalLM.from_pretrained(
    "samwit/koala-7b",
    load_in_8bit=True,
    device_map='auto',
)

pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

llm = HuggingFacePipeline(pipeline=pipe)
qa = VectorDBQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    vectorstore=vectorstore
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
Loading checkpoint shards: 100%|████████████████| 14/14 [01:21<00:00,  5.81s/it]


In [16]:
query = "What is the National Climate Change Secretariat?"
qa.run(query)

" The National Climate Change Secretariat (NCCS) is a government agency established in 2010 to develop and implement Singapore's domestic and international policies and strategies to tackle climate change. It is part of the Strategy Group that supports the Prime Minister and his Cabinet to establish priorities and strengthen strategic alignment across Government."

In [18]:
query = "How can we use hydrogen for fighting climate change?"
qa.run(query)

' Hydrogen can be used as a clean energy source and a way to store excess renewable energy. It can also be used as a chemical feedstock in various industries.'

In [19]:
query = "What is Singapore's emission targets?"
qa.run(query)

" Singapore's emission targets are to reduce emissions intensity by 36% from 2005 levels by 2030, stabilize greenhouse gas emissions with the aim of peaking around 2030, and then reduce emissions to around 60MtCO2e in 2030 after peaking emissions earlier as part of its 2030 NDC."

#### QA with source

In [24]:
from langchain.chains import VectorDBQAWithSourcesChain

qa_with_sources = VectorDBQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    vectorstore=vectorstore
)

In [26]:
qa_with_sources(query)

Input length of input_ids is 2208, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


{'question': "What is Singapore's emission targets?",
 'answer': ' Singapore',
 'sources': ''}